In [148]:
import pandas as pd
import numpy as np

ФИЛЬТРАЦИЯ DF ПО ТОВАРАМ-ИНДИКАТОРАМ В СООТВЕТСТВИИ С НАИБОЛЕЕ ПОПУЛЯРНЫМИ РАЗМЕРАМИ ПРЕДМЕТОВ

In [149]:
# читаю файл для получения списка ссылок
urls = pd.read_csv('links.csv')
urls = urls['Ссылка'].tolist()

# из ecom-файла достаю все данные и сохраняю их в DataFrame
df1 = pd.read_csv('ecom.csv')
df1 = df1.drop_duplicates()

# вытаскиваю только наши ссылки для дальнейшего мониторинга
links = []
for url in urls:
    try:
        if '585zolotoy' in url:
            links.append(url)
    except:
        continue

# извлекаю из ссылок артикулы
urls = []
for link in links:
    link = link.replace('https://www.585zolotoy.ru/catalog/products/', '')
    if '/' in link:
        link = link.replace('/', '')
    urls.append(int(link))

# немного фильтрации по отслеживаемым позициям
b = df1[(df1['article'] == 1001247)]
df = df1.query("article.isin(@urls)")
rings = df[((df['type1'] == 'КОЛЬЦО ОБРУЧАЛЬНОЕ') & (df['size'] == 17.5)) | ((df['type1'] == 'КОЛЬЦО') & (df['size'] == 17))].drop_duplicates(['article'])
chains = df[(df['type1'] == 'ЦЕПЬ') & (df['size'] == 45)].drop_duplicates(['article'])
bracelets = df[(df['type1'] == 'БРАСЛЕТ') & (df['size'] == 18)].drop_duplicates(['article'])
earrings = df[(df['type1'] == 'СЕРЬГИ')].drop_duplicates(['article'])
earrings_kongo = df[((df['type1'] == 'СЕРЬГИ-КОНГО') & (df['size'] == 1.5))]
other = df['type1'][(df['type1'] != 'КОЛЬЦО ОБРУЧАЛЬНОЕ') &
                    (df['type1'] != 'КОЛЬЦО') &
                    (df['type1'] != 'ЦЕПЬ') &
                    (df['type1'] != 'БРАСЛЕТ') &
                    (df['type1'] != 'СЕРЬГИ') &
                    (df['type1'] != 'СЕРЬГИ-КОНГО')
                    ].unique().tolist()
df = df.query("type1.isin(@other)")
df = df.drop_duplicates(['article'])

# формирую итоговый DataFrame с товарами-индикаторами 
res = pd.concat([df, rings, chains, bracelets, earrings, earrings_kongo, b], axis=0)
res = res[['article', 'size', 'discount_price', 'label_template']]

In [150]:
res = res.sort_values(by='article')

In [151]:
res = res.reset_index()

In [152]:
# преобразую столбец со скидками в числовой формат
def discount(price):
    try:
        if '-23%' in price:
            return 0.77
        elif '-10%' in price:
            return 0.90
        elif '-15%' in price:
            return 0.85
        return 0
    except:
        return 1
    
res['label_template'] = res['label_template'].apply(discount)

In [153]:
res.head()

,index,article,size,discount_price,label_template
0,155333,1001247,NaN,882.0,0.90
1,226814,1037699,NaN,3990.0,1.00
2,148174,1044635,NaN,29592.0,0.77
3,190140,1058491,1.5,4014.0,1.00
4,315142,1113845,NaN,5166.0,0.90


In [154]:
# создаю новый датафрейм для соблюдения порядка в изначальной таблице
df = pd.DataFrame(columns= ['article', 'size', 'discount_price', 'label_template'])

In [155]:
# перебираю элементы для получения их в порядке первоначального списка
for url in urls:
    id = url
    c = res.loc[res['article'] == url]
    df = pd.concat([df, c], ignore_index=True, sort=False)
    
# удаляю колонку с индексами, перетёкшими сюда из первоначального датафрейма
df.drop(columns=['index'], inplace= True)

C:\Users\Trenkin.Sergey\AppData\Local\Temp\ipykernel_19036\252233924.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, c], ignore_index=True, sort=False)


In [156]:
df 

,article,size,discount_price,label_template
0,2666732,17.0,4464.0,0.77
1,2002928,17.5,13950.0,1.00
2,4978904,17.5,8784.0,0.77
3,2002944,17.5,18342.0,1.00
4,2004212,17.5,23904.0,1.00
5,4484504,NaN,4248.0,0.77
6,1480098,NaN,6372.0,1.00
7,1058491,1.5,4014.0,1.00
8,1975092,17.0,18000.0,0.90
9,1595230,17.0,6990.0,1.00
